Los Angeles Crime 2020 - Present
===================
**`Project Goal`**
------------


Frame the Problem and Look at the Big Picture
=====================================

1. **Define the objective in business terms:** 

2. **How will your solution be used?**

3. **What are current solutions (if any)?** 

4. **How should you frame this problem?**

5. **How should performance be measured? Is the performance measure aligned with the business objective?** 

6. **What would be the minimum performance needed to reach the business objective?**

7. **What are comparable problems? Can you reuse experience or tools?** 

8. **Is human expertise available?**

9. **How would you solve the problem manually?** 

10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 